# Context Manager

A context manager is a type of function that sets up a context for your code to run in, runs your code, and then removes the context. 

In [ ]:
# Example

with open('my_file.txt') as my_file:
    text = my_file.read()
    length = len(text)

print(f'The file is {length} characters long.')

Any code that you want to run inside the context that the context manager created needs to be indented. When the indented block is done, the context manager gets a chance to clean up anything that it needs to, like when the `open()` context manager closed the file. 

There are two ways to define a context manager in Python: by using a class that has special `__enter__()` and `__exit__()` methods or by decorating a certain kind of function. 

### How to create a context manager (function based)

1. Define a function.
2. (optional) Add any set up code your context needs.
3. Use the "yield" keyword.
4. (optional) Add any teardown code your context needs.
5. Add the `@contextlib.contextmanager` decorator.

In [ ]:
@contextlib.contextmanager
defmy_context():
    # Add any set up code you need
    yield
    # Add any teardown code you need

### The "yield" keyword

The "yield" keyword may also be new to you. When you write this word, it means that you are going to return a value, but you expect to finish the rest of the function at some point in the future. The ability for a function to yield control and know that it will get to finish running later is what makes context managers so useful.

The value that your context manager yields can be assigned to a variable in the `with` statement by adding `as <variable name>`. Here, we've assigned the value 42 that `my_context()` yields to the variable `foo`. By running this code, you can see that after the context block is done executing, the rest of the my_context() function gets run, printing "goodbye". 

In [ ]:
@contextlib.contextmanager
def my_context():
    print('hello')
    yield 42 
    print('goodbye')
    
with my_context() as foo:
    print('foo is {}'.format(foo))

This setup/teardown behavior allows a context manager to hide things like connecting and disconnecting from a database so that a programmer using the context manager can just perform operations on the database without worrying about the underlying details. 

In [ ]:
@contextlib.contextmanager
def database(url):
    # set up database connection  
    db = postgres.connect(url)
    
    yield db
    
    # tear down database connection  
    db.disconnect()

In [ ]:
url = 'http://datacamp.com/data'
with database(url) as my_db:  
    course_list = my_db.execute('SELECT * FROM courses'  )

In [ ]:
@contextlib.contextmanager
def database(url):
    # set up database connection  
    db = postgres.connect(url)
    
    yield db      # yields an specific value
    
    # tear down database connection  
    db.disconnect()
    
url = 'http://datacamp.com/data'
with database(url) as my_db:  
    course_list = my_db.execute('SELECT * FROM courses'  )

In [ ]:
@contextlib.contextmanager
def in_dir(path):
    # save current working directory  
    old_dir = os.getcwd()
    
    # switch to new working directory  
    os.chdir(path)
    
    yield       # does not yield an specific value
    
    # change back to previous
    # working directory  
    os.chdir(old_dir)
    
with in_dir('/data/project_1/'):  
    project_files = os.listdir()

In [ ]:
@contextlib.contextmanager                       # Element 3
def timer():                                     # Element 1
  """Time the execution of a context block.

  Yields:
    None
  """
  start = time.time()

  yield                                          # Element 2
  
  end = time.time()
  print('Elapsed: {:.2f}s'.format(end - start))

with timer():
  print('This should take approximately 0.25 seconds')
  time.sleep(0.25)

Notice that the three elements of a context manager are all here: a function definition, a yield statement, and the @contextlib.contextmanager decorator. It's also worth noticing that timer() is a context manager that does not return an explicit value, so yield is written by itself without specifying anything to return.

In [ ]:
@contextlib.contextmanager
def open_read_only(filename):
  """Open a file in read-only mode.

  Args:
    filename (str): The location of the file to read

  Yields:
    file object
  """
  read_only_file = open(filename, mode='r')
  # Yield read_only_file so it can be assigned to my_file
  yield read_only_file
  # Close read_only_file
  read_only_file.close()

with open_read_only('my_file.txt') as my_file:
  print(my_file.read())

This function is an example of a context manager that _does_ return a value, so we write yield read_only_file instead of just yield. Then the read_only_file object gets assigned to my_file in the with statement so that whoever is using your context can call its .read() method in the context block.

## Nested context

In [ ]:
# Use the "stock('NVDA')" context manager
# and assign the result to the variable "nvda"
with stock('NVDA') as nvda:
  # Open "NVDA.txt" for writing as f_out
  with open('NVDA.txt', 'w') as f_out:
    for _ in range(10):
      value = nvda.price()
      print('Logging ${:.2f} for NVDA'.format(value))
      f_out.write('{:.2f}\n'.format(value))

## Handling errors

In [ ]:
def in_dir(directory):
  """Change current working directory to `directory`,
  allow the user to run some code, and change back.

  Args:
    directory (str): The path to a directory to work in.
  """
  current_dir = os.getcwd()
  os.chdir(directory)

  # Add code that lets you handle errors
  try:
    yield
  # Ensure the directory is reset,
  # whether there was an error or not
  finally:
    os.chdir(current_dir)

## When to create a context manager